In [8]:
from dataset1 import *
import numpy as np
from scipy.linalg import solve_triangular
from numpy.linalg import eigh
import torch
from hadamard_transform import hadamard_transform
import time
import pandas as pd
import seaborn as sns
#test

## Define functions

In [9]:

def getError(A,B):
    return np.linalg.norm(A-B,ord='nuc')/np.linalg.norm(A,ord='nuc')

def fjlt(A, D, indices, direction):
    """
    Applies the Fast Johnson-Lindenstrauss Transform (FJLT) to a square matrix.

    Parameters:
    - A: Input square matrix (NumPy array of size n x m).
    - D: Diagonal vector of +1 and -1 (NumPy array of size n).
    - indices: Indices of the selected rows or columns for subsampling (NumPy array of size l).
    - direction: 'left' to apply FJLT from the left (on rows) or 'right' for the transpose case (on columns).

    Returns:
    - A_proj: Transformed matrix after applying FJLT and subsampling.
    """
    n, m = A.shape
    l = indices.shape[0]  # Sketch size
    if direction == 'left':  # FJLT applied to the rows of A
        assert len(D) == n, "The length of D must match the number of rows in A."
        assert (n & (n - 1) == 0), "The number of rows in A must be a power of 2."
        # Multiply each row of A by the corresponding element in D
        OmegaA = A * D[:, np.newaxis]
        # Apply the Fast Hadamard Transform to the rows
        OmegaA = hadamard_transform(torch.tensor(OmegaA.T)).numpy().T
        # Subsample the rows and scale
        OmegaA = OmegaA[indices, :] * np.sqrt(n / l)
        return np.ascontiguousarray(OmegaA)
    elif direction == 'right':  # FJLT applied to the columns of A
        # Multiply each column of A by the corresponding element in D
        AOmega = A * D[np.newaxis, :]
        # Apply the Fast Hadamard Transform to the columns
        AOmega = hadamard_transform(torch.tensor(AOmega)).numpy()
        # Subsample the columns and scale
        AOmega = AOmega[:, indices] * np.sqrt(n / l)
        return np.ascontiguousarray(AOmega)

def getLowRank(A, rank, sketch_dim, sketch_method='gaussian', good_conditionned=True, print_times=False):
    """
    Compute a low-rank approximation of the matrix A using sketching, 
    eigenvalue decomposition, and truncation. Optionally prints the time taken for each major operation.

    Parameters:
        A (numpy.ndarray): Original matrix (size m x n).
        rank (int): Desired rank for the approximation.
        sketch_dim (int): Dimension of the sketching subspace.
        sketch_method (str): Method used for sketching. Options are:
                             - 'gaussian': Uses a Gaussian random matrix for sketching.
                             - 'fjlt': Uses the Fast Johnson-Lindenstrauss Transform.
        good_conditionned (bool): If True, uses Cholesky decomposition for better conditioning. 
                                  Otherwise, uses eigenvalue decomposition.
        print_times (bool): If True, prints the time taken for each major operation.

    Returns:
        A_nyst_k (numpy.ndarray): Low-rank approximation of A (size m x m).
    """
    m = A.shape[0]
    B = None
    C = None

    start_time = time.time()

    # Apply sketching based on the selected method
    if sketch_method == 'fjlt':  # Use Fast Johnson-Lindenstrauss Transform for sketching
        D = np.random.choice([1, -1], size=m)
        indices = np.random.choice(m, size=sketch_dim, replace=False)
        sketch_start = time.time()
        C = fjlt(A, D, indices, direction='right')
        B = fjlt(C, D, indices, direction='left')
        if print_times:
            print(f"FJLT sketching completed in {time.time() - sketch_start:.6f} seconds.")

    elif sketch_method == 'gaussian':  # Use Gaussian random matrix for sketching
        Omega = np.random.normal(size=(m, sketch_dim))
        sketch_start = time.time()
        C = A @ Omega
        B = Omega.T @ C
        if print_times:
            print(f"Gaussian sketching completed in {time.time() - sketch_start:.6f} seconds.")
    else:
        raise ValueError("Méthode de sketching non reconnue. Choisissez 'gaussian' ou 'fjlt'.")

    if good_conditionned:
        # Use Cholesky decomposition
        cholesky_start = time.time()
        L = np.linalg.cholesky(B)
        if print_times:
            print(f"Cholesky decomposition completed in {time.time() - cholesky_start:.6f} seconds.")
        
        triangular_start = time.time()
        Z = solve_triangular(L, C.T, lower=True).T
        if print_times:
            print(f"Triangular solve completed in {time.time() - triangular_start:.6f} seconds.")
    else:
        # Use eigenvalue decomposition
        eigh_start = time.time()
        eig_v, V = eigh(B)
        if print_times:
            print(f"Eigenvalue decomposition completed in {time.time() - eigh_start:.6f} seconds.")
        
        truncate_start = time.time()
        truncate = eig_v > 0
        eig_v = eig_v[truncate]
        V = V[:, truncate]
        Z = C @ (V * (1 / np.sqrt(eig_v)))
        if print_times:
            print(f"Eigenvalue truncation and normalization completed in {time.time() - truncate_start:.6f} seconds.")

    qr_start = time.time()
    Q, R = np.linalg.qr(Z)
    if print_times:
        print(f"QR decomposition completed in {time.time() - qr_start:.6f} seconds.")

    svd_start = time.time()
    U, Sigma, _ = np.linalg.svd(R)
    if print_times:
        print(f"SVD completed in {time.time() - svd_start:.6f} seconds.")

    approx_start = time.time()
    U_hat = Q @ U[:, :rank]
    A_nyst_k = (U_hat * Sigma[:rank]**2) @ U_hat.T
    if print_times:
        print(f"Low-rank approximation construction completed in {time.time() - approx_start:.6f} seconds.")

    total_time = time.time() - start_time
    if print_times:
        print(f"Total computation time: {total_time:.6f} seconds.")
    error = getError(A,A_nyst_k)
    print(f"Error :{error}".format(error))

    return A_nyst_k, error 

## Tests

In [10]:
n = 2048
A = getExpMatrix(n,R=10)
print(A.shape)

(2048, 2048)


In [11]:
getLowRank(A,rank=128,sketch_dim=128,sketch_method='gaussian',good_conditionned=False,print_times=True)

Gaussian sketching completed in 0.016633 seconds.
Eigenvalue decomposition completed in 0.006657 seconds.
Eigenvalue truncation and normalization completed in 0.002259 seconds.
QR decomposition completed in 0.027455 seconds.
SVD completed in 0.004365 seconds.
Low-rank approximation construction completed in 0.013040 seconds.
Total computation time: 0.079953 seconds.
Error :1.2155091862342842e-14


(array([[ 1.00000000e+00,  1.34614542e-15, -1.00613962e-16, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 1.33920652e-15,  1.00000000e+00,  7.89299182e-16, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-7.28583860e-17,  7.19910243e-16,  1.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]]),
 1.2155091862342842e-14)

In [ ]:
# getLowRank(A,rank=128,sketch_dim=128,sketch_method='fjlt',good_conditionned=False,print_times=True)

FJLT sketching completed in 0.144048 seconds.
Eigenvalue decomposition completed in 0.008370 seconds.
Eigenvalue truncation and normalization completed in 0.002851 seconds.
QR decomposition completed in 0.032139 seconds.
SVD completed in 0.006719 seconds.
Low-rank approximation construction completed in 0.010890 seconds.
Total computation time: 0.209292 seconds.
Error :3.924891156083275e-06


(array([[ 9.99998616e-01,  1.38412188e-06, -1.38412185e-06, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 1.38412188e-06,  9.99998616e-01,  1.38412190e-06, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-1.38412185e-06,  1.38412190e-06,  9.99998616e-01, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]]),
 3.924891156083275e-06)

## Section 2

We'll compute the error with respect to three matrices. 
1. polynomial decay (dimension 1024 * 1024) locates: 
2. exp decay locates:
3. ../Dataset/A_3_dataset_2_mnist.npy 
    denotes the matrix obtained from MNIST, with c=100
4. ../... mnist.npy with c = 10.

All four matrices are of dimension $1024 \times 1024$

In [13]:
# Loading matrices

loadMatrix_A_1 = np.load("../Dataset/A_1_polyDecayMatrix.npy")
print("matrix A_1")
print(loadMatrix_A_1.shape)
print(loadMatrix_A_1[:10,:5])

loadMatrix_A_2 = np.load("../Dataset/A_2_expDecayMatrix.npy")
print("matrix A_2")
print(loadMatrix_A_2.shape)
print(loadMatrix_A_2[:10,:5])

loadMatrix_A_3 = np.load("../Dataset/A_3_dataset_2_mnist.npy")
print("matrix A_3")
print(loadMatrix_A_3.shape)
print(loadMatrix_A_3[:10,:5])

loadMatrix_A_32 = np.load("../Dataset/A_3_dataset_2_mnist_C=10.npy")
print("matrix A_32")
print(loadMatrix_A_32.shape)
print(loadMatrix_A_32[:10,:5])



matrix A_1
(1024, 1024)
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
matrix A_2
(1024, 1024)
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
matrix A_3
(1024, 1024)
[[1.         0.99127504 0.98824289 0.99001237 0.98907651]
 [0.99127504 1.         0.98715919 0.98829865 0.98930957]
 [0.98824289 0.98715919 1.         0.9898814  0.98994457]
 [0.99001237 0.98829865 0.9898814  1.         0.99126262]
 [0.98907651 0.98930957 0.98994457 0.99126262 1.        ]
 [0.98979753 0.98903161 0.98938555 0.99024257 0.99195122]
 [0.99100754 0.98756737 0.98972827 0.991512   0.9925866 ]
 [0.99278471 0.98967594 0.98934556 0.98962114 0.98935745]
 [0.99136177 0.98870132 0.99122707 0.99290202 0.9940263 ]
 [0.98989935 0.98824599 0.99277739 0.9911983

In [16]:
from tqdm import tqdm  # For progress tracking

def collect_low_rank_errors(
    A, 
    sketch_dims=[2, 4, 8, 16, 32, 64, 128], 
    is_good_conditionned=False, 
    sketch_method='gaussian', 
    output_file='Section_2_test_1_for_loadMatrixA_1.xlsx',
    matrix_name="MatrixA"
):
    """
    Computes errors for various ranks and sketch dimensions using the `getLowRank` function.

    Parameters:
    - A (np.ndarray): Input matrix.
    - sketch_dims (list of int): List of sketch dimensions to test.
    - is_good_conditionned (bool): Whether to use good-conditioned matrices in `getLowRank`.
    - sketch_method (str): Method for sketching ('gaussian' or other supported by `getLowRank`).
    - output_file (str): Path to the file where results are saved (.csv or .xlsx).
    - matrix_name (str): Name or identifier of the input matrix.

    Saves:
    - Results to a file with columns: "Sketch Dimension", "Approximation Rank", "Error Value",
      "Sketch Method", "Matrix Name".
    """
    print(A.shape)
    results = []
    
    # Iterate over each sketch_dim
    for sketch_dim in tqdm(sketch_dims, desc="Sketch Dimensions Progress", unit="dim"):
        ranks = list(range(0, sketch_dim + 1, 10))  # Generate ranks from 0 with step 5 up to sketch_dim
        for rank in ranks:
            # Compute the error for each rank
            _, error = getLowRank(
                A=A, 
                rank=rank, 
                sketch_dim=sketch_dim, 
                sketch_method=sketch_method, 
                good_conditionned=is_good_conditionned
            )
            # Append the result
            results.append({
                'Sketch Dimension': sketch_dim,
                'Approximation Rank': rank,
                'Error Value': error,
                'Sketch Method': sketch_method,
                'Matrix Name': matrix_name
            })
    
    # Convert results to a DataFrame for easier manipulation
    df = pd.DataFrame(results)

    # Save results to file
    if output_file.endswith('.csv'):
        df.to_csv(output_file, index=False)
    elif output_file.endswith('.xlsx'):
        df.to_excel(output_file, index=False, engine='openpyxl')
    else:
        raise ValueError("Output file must be either .csv or .xlsx.")
    
    print(f"Results saved to {output_file}.")



In [19]:
# collect_low_rank_errors(loadMatrix_A_1)
testList1 = [25, 50, 75, 100, 125, 150]
testList2 = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
testList3 = [25, 50, 75, 100, 125, 150, 175, 200]

collect_low_rank_errors(A = loadMatrix_A_1, 
                        sketch_dims = [25, 50, 75, 100, 125, 150, 175, 200],  
                        is_good_conditionned=True, 
                        sketch_method='gaussian', 
                        output_file='../Dataset/Section_2_test_3_Gaussian_for_loadMatrixA_1.csv', matrix_name="loadMatrix_A_1")

collect_low_rank_errors(A = loadMatrix_A_2, 
                        sketch_dims = testList,  
                        is_good_conditionned=False, 
                        sketch_method='gaussian', 
                        output_file='../Dataset/Section_2_test_3_Gaussian_for_loadMatrixA_2.csv', matrix_name="loadMatrix_A_2")

collect_low_rank_errors(A = loadMatrix_A_3, 
                        sketch_dims = [25, 50, 75, 100, 125, 150, 175, 200],  
                        is_good_conditionned=False, 
                        sketch_method='gaussian', 
                        output_file='../Dataset/Section_2_test_3_Gaussian_for_loadMatrixA_3.csv', matrix_name="loadMatrix_A_3")

collect_low_rank_errors(A = loadMatrix_A_32, 
                        sketch_dims = [25, 50, 75, 100, 125, 150, 175, 200],  
                        is_good_conditionned=False, 
                        sketch_method='gaussian', 
                        output_file='../Dataset/Section_2_test_3_Gaussian_for_loadMatrixA_32.csv', matrix_name="loadMatrix_A_3")

collect_low_rank_errors(A = loadMatrix_A_1, 
                        sketch_dims = [25, 50, 75, 100, 125, 150, 175, 200],  
                        is_good_conditionned=True, 
                        sketch_method='fjlt', 
                        output_file='../Dataset/Section_2_test_4_fjlt_for_loadMatrixA_1.csv', matrix_name="loadMatrix_A_1")

collect_low_rank_errors(A = loadMatrix_A_2, 
                        sketch_dims = [25, 50, 75, 100, 125, 150, 175, 200],  
                        is_good_conditionned=False, 
                        sketch_method='fjlt', 
                        output_file='../Dataset/Section_2_test_4_fjlt_for_loadMatrixA_2.csv', matrix_name="loadMatrix_A_2")

collect_low_rank_errors(A = loadMatrix_A_3, 
                        sketch_dims = [25, 50, 75, 100, 125, 150, 175, 200],  
                        is_good_conditionned=False, 
                        sketch_method='fjlt', 
                        output_file='../Dataset/Section_2_test_4_fjlt_for_loadMatrixA_3.csv', matrix_name="loadMatrix_A_3")

collect_low_rank_errors(A = loadMatrix_A_32, 
                        sketch_dims = [25, 50, 75, 100, 125, 150, 175, 200],  
                        is_good_conditionned=False, 
                        sketch_method='fjlt', 
                        output_file='../Dataset/Section_2_test_4_fjlt_for_loadMatrixA_32.csv', matrix_name="loadMatrix_A_32")


(1024, 1024)


Sketch Dimensions Progress:   0%|          | 0/8 [00:00<?, ?dim/s]

Error :1.0
Error :0.960772448919279


Sketch Dimensions Progress:  12%|█▎        | 1/8 [00:02<00:15,  2.17s/dim]

Error :0.9477751787564648
Error :1.0
Error :0.945709004216863
Error :0.9287808404175188
Error :0.9136843258015594
Error :0.9040175036572112


...

Error :0.2648849341739228
Error :1.0
Error :0.4423111278049151
Error :0.37813234833308895
Error :0.3457365035981818
Error :0.32234633283191655
Error :0.3077213296094297
Error :0.29693764327538086
Error :0.2869815978751995
Error :0.2805987477453641
Error :0.2752198548731282
Error :0.2685478096988795
Error :0.2655631528373714
Error :0.259937345314048
Error :0.25850345421183585
Error :0.25466924969084853
Error :0.24978515415065078
Error :0.2486658726731785


Sketch Dimensions Progress:  88%|████████▊ | 7/8 [01:12<00:13, 13.78s/dim]

Error :0.24673348280808957
Error :1.0
Error :0.440259205966917
Error :0.37447785715735804
Error :0.3392182432153556
Error :0.3168355235340421
Error :0.301895986725637
Error :0.28943538194889584
Error :0.27943918475682966
Error :0.2723572240924201
Error :0.26595044092839026
Error :0.26009096612751537
Error :0.2547632622878583
Error :0.2493346006713285
Error :0.24690018983007764
Error :0.24348498691639234
Error :0.23914751276382837
Error :0.2361788983115978
Error :0.23521218456190301
Error :0.23129575839463504
Error :0.22878378281242467


Sketch Dimensions Progress: 100%|██████████| 8/8 [01:32<00:00, 11.61s/dim]

Error :0.22724330245067237
Results saved to ../Dataset/Section_2_test_4_fjlt_for_loadMatrixA_32.csv.


### Diagram 

In the following parts, we'll draw diagrams. 

In [28]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

def plot_statistical_diagram_with_logscale(csv_file, save_path, include_labels=True):
    """
    Plots a statistical diagram comparing error values with approximation rank for different sketch dimensions.
    Uses a logarithmic scale for error values.
    
    Parameters:
        csv_file (str): Path to the .csv file containing the data.
        save_path (str): File path to save the resulting plot.
        include_labels (bool): If True, includes axis labels and a legend. If False, no labels or legend will be displayed.
    """
    # Load the CSV data
    data = pd.read_csv(csv_file)
    
    # Initialize the Seaborn style
    sns.set(style="whitegrid")
    
    # Create the line plot
    plt.figure(figsize=(10, 6))
    sns.lineplot(
        data=data,
        x="Approximation Rank",
        y="Error Value",
        hue="Sketch Dimension",
        style="Sketch Dimension",
        markers=True,
        dashes=False,
        palette="deep"
    )
    
    # Use log scale for the y-axis (Error Value)
    plt.yscale('log')
    
    # Set axis labels and title if needed
    if include_labels:
        plt.title("Error (Log Scale) vs Approximation Rank for Different Sketch Dimensions", fontsize=14)
        plt.xlabel("Approximation Rank", fontsize=12)
        plt.ylabel("Error Value (Log Scale)", fontsize=12)
        plt.legend(title="Sketch Dimension", fontsize=10)
    else:
        plt.axis("off")
    
    # Save the plot
    plt.tight_layout()
    plt.savefig(save_path, dpi=300)
    plt.close()
    print(f"Plot saved at {save_path}")

# Example usage:
# plot_statistical_diagram_with_logscale("Section_2_test_3_Gaussian_for_loadMatrixA_1.csv", "error_vs_rank_logscale.png", include_labels=True)


In [31]:
plot_statistical_diagram_with_logscale("../Dataset/Section_2_test_3_Gaussian_for_loadMatrixA_1.csv", "error_vs_rank_A_1_Gaussian.png", include_labels=True)
plot_statistical_diagram_with_logscale("../Dataset/Section_2_test_3_Gaussian_for_loadMatrixA_2.csv", "error_vs_rank_A_2_Gaussian.png", include_labels=True)
plot_statistical_diagram_with_logscale("../Dataset/Section_2_test_3_Gaussian_for_loadMatrixA_3.csv", "error_vs_rank_A_3_Gaussian.png", include_labels=True)
plot_statistical_diagram_with_logscale("../Dataset/Section_2_test_3_Gaussian_for_loadMatrixA_32.csv", "error_vs_rank_A_32_Gaussian.png", include_labels=True)
plot_statistical_diagram_with_logscale("../Dataset/Section_2_test_4_fjlt_for_loadMatrixA_1.csv", "error_vs_rank_A_1_fjlt.png", include_labels=True)
plot_statistical_diagram_with_logscale("../Dataset/Section_2_test_4_fjlt_for_loadMatrixA_2.csv", "error_vs_rank_A_2_fjlt.png", include_labels=True)
plot_statistical_diagram_with_logscale("../Dataset/Section_2_test_4_fjlt_for_loadMatrixA_3.csv", "error_vs_rank_A_3_fjlt.png", include_labels=True)
plot_statistical_diagram_with_logscale("../Dataset/Section_2_test_4_fjlt_for_loadMatrixA_32.csv", "error_vs_rank_A_32_fjlt.png", include_labels=True)


Plot saved at error_vs_rank_A_1_Gaussian.png
Plot saved at error_vs_rank_A_2_Gaussian.png
Plot saved at error_vs_rank_A_3_Gaussian.png
Plot saved at error_vs_rank_A_32_Gaussian.png
Plot saved at error_vs_rank_A_1_fjlt.png
Plot saved at error_vs_rank_A_2_fjlt.png
Plot saved at error_vs_rank_A_3_fjlt.png
Plot saved at error_vs_rank_A_32_fjlt.png
